In [11]:
import sys
sys.path.insert(1, '/media/cinthia/Dados/Mestrado/mv-text-summarizer')

import pandas as pd
import json
import spacy
import nltk

from sumeval.metrics.rouge import RougeCalculator
from bs4 import BeautifulSoup
from pysbd.utils import PySBDFactory

from src import preprocess
from src import extract_features
from src import tokenizer
from src import create_features_df
from src import transform_data

rouge = RougeCalculator(stopwords=True, lang="en")

In [2]:
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('punkt')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/cinthia/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /home/cinthia/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /home/cinthia/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt to /home/cinthia/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
!python -m spacy download en_core_web_sm
nlp_sm = spacy.load('en_core_web_sm')

     |████████████████████████████████| 11.1 MB 2.3 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [4]:
!python -m spacy download en_core_web_md
nlp_md = spacy.load('en_core_web_md')

     |████████████████████████████████| 95.4 MB 14.2 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')


In [5]:
path_base = "../sample"
files = os.listdir(path_base)

In [6]:
texts = []
for file in files:
  texts.append(json.load(open('{}/{}'.format(path_base, file))))

In [7]:
introductions = [preprocess.format_intro(i.get('sec_introduction')) for i in texts]
abstract = [preprocess.format_intro(i.get('sec_abstract')) for i in texts]
article_keyword = [i.get('sec_keyword') for i in texts]

In [8]:
result = []
all_scores_df =[ ]
all_labels = []

select_features = True
valid = True


for i in range(1):

  print(i)
  number_text = i

  xml = preprocess.format_xml(str(introductions[number_text]))
  text = preprocess.format_text(str(introductions[number_text]), post_processing=False)
  reference = preprocess.format_text(str(abstract[number_text]), post_processing=True)

  bibs = extract_features.get_citations(xml)
  text = preprocess.exclude_bib(text, bibs)
  text = preprocess.format_text(text, post_processing=True)

  soup = BeautifulSoup(text)
  text = soup.get_text()

  sentences = tokenizer.split_sentences([text])
  sentences = list(map(str, sentences[0]))
  sentences = preprocess.format_sentences(sentences)

  try: 
    if select_features:
      features = create_features_df.main(sentences, xml, article_keyword,number_text, nlp_sm, nlp_md)
      features_df = create_features_df.format_df (sentences, features)
      result.append(features_df)

    if valid:
      sentences_ref = tokenizer.split_sentences([reference])
      sentences_ref = list(map(str, sentences_ref[0]))
      scores_df, label = transform_data.main_create_label(sentences, sentences_ref, rouge)
      scores_df['label'] = label

      all_scores_df.append(scores_df)
      all_labels.append(label)

  except ValueError as error:
      print(repr(error))
      pass

0
P2X receptors for ATP are ligand gated cation channels and form from the homo  or heterotrimeric assembly of seven different receptor subunits (P2X1 7) to give a range of phenotypes . They are expressed widely in the nervous system and contribute to gut peristalsis , neuronal hypoxia reflexes , mechanotransduction , and pain sensation (; ; ; ). P2X receptors do not contain common consensus ATP binding sequences e.g. the Walker motif  and the structure and organisation of the binding site is unclear. A model incorporating four co ordinates for ATP action at the P2X1 receptor; with positively charged lysines K68 and K309 co ordinating binding of the phosphate tail of ATP, and the adenine ring sandwiched between the domains incorporating phenylalanine (F) residues F185, T186 and N290, F291, R292 with R292 co ordinating the binding of the α phosphate has been proposed . This model is supported by mutagenesis data from P2X2  and P2X7 receptors . In the model the residues associated with A

In [9]:
scores_df

,rouge_1,rouge_2,rouge_l,label
0,0.358974,0.216216,0.358974,0
1,0.062500,0.000000,0.062500,0
2,0.615385,0.333333,0.461538,1
3,0.258065,0.066667,0.161290,0
4,0.105263,0.000000,0.105263,0
5,0.260870,0.153846,0.243902,0
6,0.258065,0.090909,0.166667,0
7,0.129032,0.000000,0.105263,0
8,0.250000,0.086957,0.166667,0
9,0.456140,0.254545,0.192308,1


In [12]:
for i in range(len(result)):
  df = pd.DataFrame(result[i])
  df.to_csv("../result/features_{}.csv".format(i), index=False)

In [13]:
for i in range(len(all_scores_df)):
  df = pd.DataFrame(all_scores_df[i])
  df.to_csv("../result/score_{}.csv".format(i), index=False)